In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import torch
from tqdm import tqdm
from transformers import BertTokenizer
from parallel_processor import process_data

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('/hy-nas/LOTClass/datasets/agnews/train.txt', 'r') as fin:
    data = [item.replace('\n', '') for item in fin.readlines()]

In [21]:
with open('/hy-nas/LOTClass/datasets/agnews/train_labels.txt', 'r') as fin:
    label = [int(item.replace('\n', '')) for item in fin.readlines()]

In [4]:
# dic = torch.load('/hy-nas/LOTClassdatasets/agnews/category_vocab.pt', 'rb')

In [5]:
tker = BertTokenizer.from_pretrained('bert-base-uncased')

In [59]:
with open('/hy-nas/LOTClass/datasets/agnews/cate_vocab_with_weight.pkl', 'rb') as fin:
    dic = pkl.load(fin)
dic

{'politics': [['tension', 0.8971363306045532],
  ['personnel', 0.8825752139091492],
  ['style', 0.8620188236236572],
  ['leaders', 0.857577919960022],
  ['fight', 0.8542557954788208],
  ['school', 0.8454718589782715],
  ['threat', 0.8417034149169922],
  ['weekend', 0.8352767825126648],
  ['stay', 0.8344505429267883],
  ['response', 0.8317400813102722],
  ['draft', 0.8299174308776855],
  ['attempt', 0.829717755317688],
  ['flag', 0.8244023323059082],
  ['moment', 0.8197098970413208],
  ['sept', 0.8189804553985596],
  ['reporters', 0.8178490996360779],
  ['dec', 0.8154200315475464],
  ['scare', 0.8147599101066589],
  ['unity', 0.8140192627906799],
  ['outcome', 0.8133175373077393],
  ['businessman', 0.8123329281806946],
  ['mourners', 0.8111353516578674],
  ['protests', 0.8110515475273132],
  ['amid', 0.8096731305122375],
  ['dialogue', 0.8096408843994141],
  ['citizens', 0.8094082474708557],
  ['trouble', 0.8091041445732117],
  ['italian', 0.8089165687561035],
  ['leadership', 0.8082014

In [11]:
# with open('dbpedia_lotclass_dic.pkl', 'rb') as fin:
#     dic = pkl.load(fin)

In [61]:
# def pseudo_label(x, vocab):
#     score = {k: 0 for k in vocab}
#     for k, v in vocab.items():
#         cnt = 0
#         for w in v:
#             if w[0] in x:
#                 score[k] += w[1] ** 2
#                 cnt += 1
#         if cnt == 0:
#             score[k] = 0
#         else:
#             score[k] /= (cnt ** 0.5)
#     return list(score.values())

# np.argmax(list(pseudo_label(data[-100000], dic))), data[-100000]

In [68]:
def pseudo_label_batch(data, vocab):
    def pseudo_label(x):
        score = {k: 0 for k in vocab}
        for k, v in vocab.items():
            cnt = 0
            for w in v:
                if w[0] in x:
                    score[k] += w[1]
                    cnt += 1
            if cnt < 4:
                score[k] = 0
            else:
                score[k] /= (cnt ** 0.5)
        return list(score.values())
#     def pseudo_label(x):
#         score = {k: 0 for k in vocab}
#         for k, v in vocab.items():
#             for w in v:
#                 if w in x:
#                     score[k] += 1
#         return list(score.values())

    plabel = []
    for doc in tqdm(data):
#         plabel.append(np.argmax(pseudo_label(doc)))
        plabel.append(pseudo_label(doc))
    return plabel

from functools import partial

pseudo_label_batch_p = partial(pseudo_label_batch, vocab=dic)

plabel = process_data(data, pseudo_label_batch_p, num_workers=60)

100%|██████████| 2000/2000 [00:01<00:00, 1071.03it/s]


In [69]:
plabel.shape, np.sum(np.sum(plabel, axis=1) == 0)

((120000, 4), 104750)

In [74]:
sele_idx = np.sum(plabel, axis=1) > 0
plabel_ = plabel[sele_idx]
plabel_.shape

(15250, 4)

In [75]:
label_ = np.array(label)[sele_idx]
np.mean(np.argmax(plabel_, axis=1) == label_)

0.6714098360655738

In [72]:
for i in tqdm(range(len(set(label_)))):
    print(i, np.mean(label_[label_ == i] == np.argmax(plabel_[label_==i], axis=1)))

100%|██████████| 4/4 [00:00<00:00, 1023.19it/s]

0 0.5395763656633222
1 0.93
2 0.9240941095655161
3 0.4089311859443631


In [73]:
from collections import Counter

for i in tqdm(range(len(set(label_)))):
    cnt = Counter(np.argmax(plabel_[label_==i], axis=1))
    print(i, cnt)

100%|██████████| 4/4 [00:00<00:00, 774.11it/s]

0 Counter({0: 484, 2: 313, 1: 62, 3: 38})
1 Counter({1: 1581, 2: 81, 0: 33, 3: 5})
2 Counter({2: 5381, 3: 411, 0: 29, 1: 2})
3 Counter({2: 3985, 3: 2793, 0: 37, 1: 15})


In [86]:
data_ = [data[i] for i, v in enumerate(sele_idx) if v == True]
len(data_)

15250

In [92]:
df = pd.DataFrame({'text': data_, 'label': label_, 'plabel': [','.join(list(map(str, i))) for i in plabel_]})
df

,text,label,plabel
0,Carlyle Looks Toward Commercial Aerospace ( Re...,2,"0.0,0.0,1.7638108134269714,0.0"
1,Saudi phone sector gets $1bn lift. A group led...,2,"0.0,0.0,1.6555531919002533,0.0"
2,"Google IPO: Type in 'confusing,' 'secrecy'. I'...",2,"0.0,0.0,1.9099247152256673,0.0"
3,Technology company sues five ex-employees . A ...,2,"0.0,0.0,1.7361441552639008,0.0"
4,"Switching Titles, if Not Gears, at Dell. Kevin...",2,"0.0,0.0,1.9076497582657077,0.0"
...,...,...,...
15245,"Despite Mega-Deals, Small Mergers Rule ( Reute...",2,"0.0,0.0,1.701511949300766,0.0"
15246,"Stocks to Hold Up, But Anxiety Over Drugs. NEW...",2,"0.0,0.0,1.7602374851703644,0.0"
15247,"Despite Mega-Deals, Small Mergers Rule. Philad...",2,"0.0,0.0,1.701511949300766,0.0"
15248,Wal-Mart Dec. Sales Still Seen Up 1-3 Pct. CHI...,2,"0.0,0.0,2.069745707581295,0.0"


In [93]:
df.to_csv('/hy-nas/LOTClass/datasets/agnews/plabel_train.csv', index=False)